In [ ]:
from modules.lstm_encoder import LSTMEncoder
from modules.lstm_decoder import LSTMDecoder
from keras.regularizers import l2
import os
import numpy as np
import pandas as pd
from IPython.display import display, Markdown, Latex, HTML, clear_output
import ipy_table
import ipywidgets as widgets

## Encoding

In [ ]:
encoder = LSTMEncoder(l2, './features/lstm/2_steps/', './convnet_weights/lstm/base/weights.19-1.13.hdf5')

In [ ]:
features_dev = encoder.encode('dev')

In [ ]:
features_dev.shape

## Classes

In [ ]:
indices = np.load('./features/lstm/2_steps/indices_dev.npy')
true_classes = np.load('./features/res_net/classes_dev_make-model.npy')[indices]
true_classnames = np.load('./features/res_net/classnames_dev_make-model.npy')[indices]
true_filenames = np.load('./features/res_net/filenames_dev.npy')[indices]

In [ ]:
true_classes

In [ ]:
true_classnames

In [ ]:
true_filenames

## Predictions

In [ ]:
def map_predictions_to_classnames(predictions, true_classes, classnames):
    classes, indices = np.unique(true_classes, return_index=True)
    
    class_to_index = {}
    for cl, index in zip(classes, indices):
        class_to_index[cl] = index
    
    predicted_classnames = list(map(lambda cl: classnames[class_to_index[cl]], predictions))
    return np.array(predicted_classnames)

In [ ]:
decoder = LSTMDecoder(None, None, features_dev, true_classes, weights_path='./convnet_weights/lstm/decoder_lstm/weights.36-1.27.hdf5')

In [ ]:
predictions = decoder.predict('dev')

In [ ]:
predicted_classnames = map_predictions_to_classnames(predictions, true_classes, true_classnames)

## Reports creation

In [ ]:
classes, indices = np.unique(true_classes, return_index=True)
classnames = true_classnames[indices]
directories = list(map(lambda f: f.split('/')[0],true_filenames[indices]))

In [ ]:
classes_report = pd.DataFrame({
    'name': classnames,
    'directory': directories,
}, index = classes)

In [ ]:
pred_classnames = list(map(lambda cl: classes_report['name'][cl], predictions))

In [ ]:
samples_report = pd.DataFrame({
    'filename': true_filenames,
    'prediction': predictions,
    'real_class': true_classes,
    'predicted_name': pred_classnames,
    'real_name': true_classnames,
    'predicted_correctly': predictions == true_classes,
})

In [ ]:
classes_count = samples_report['real_class'].value_counts()
classes_report['count'] = classes_count.sort_index()

In [ ]:
correct_count = samples_report[samples_report['predicted_correctly'] == True]['real_class'].value_counts()
classes_report['correct_predictions_count'] = correct_count.sort_index()

In [ ]:
classes_report['correct_predictions_percent'] = classes_report['correct_predictions_count'] / classes_report['count']

In [ ]:
classes_report.sort_values('count').head()

## Classes report

In [ ]:
def clear_report():
    clear_output()
    display(widgets.HBox((dropdown_sort, dropdown_direction)))

def display_classes_report(sortBy, direction):
    html = pd.DataFrame.to_html(classes_report.sort_values(sortBy, ascending=direction))
    display(HTML(html))
    
def clear_and_display_report(sortBy, direction):
    clear_report()
    display_classes_report(sortBy, direction)

In [ ]:
dropdown_sort = widgets.Dropdown(
    options=['correct_predictions_percent', 'name', 'count'],
    value='correct_predictions_percent',
    description='Sort by:',
    disabled=False,
)

dropdown_direction = widgets.Dropdown(
    options=['ascending', 'descending'],
    value='ascending',
    description='Direction:',
    disabled=False,
)

dropdown_sort.observe(lambda v: 
                          clear_and_display_report(v['new'], dropdown_direction.value == 'ascending') 
                          if v['name'] == 'value'
                          else None
                     )
dropdown_direction.observe(lambda v: 
                              clear_and_display_report(dropdown_sort.value, v['new'] == 'ascending') 
                              if v['name'] == 'value'
                              else None
                          )

clear_and_display_report(dropdown_sort.value, dropdown_direction.value == 'ascending')